In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 6 – APIs and Web Scraping

## DSC 80, Spring 2022

### Due Date: Monday, May 9th at 11:59 PM

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying `lab.py` file will be tested (a la DSC 20),
2. The notebook may be graded (if it contains free response questions or asks you to draw plots).

**Do not change the function names in the `lab.py` file!**
- The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name.
- If you changed something you weren't supposed to, just use git to revert! Ask us if you need help with this, or google around for `git revert`.

**Tips for working in the notebook**:
- The notebooks serve to present the questions and give you a place to present your results for later review.
- The notebooks in *lab assignments* are not graded (only the `lab.py` file is submitted and graded).
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file.

**Tips for developing in the `lab.py` file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional helper functions to solve the lab! 
- Always document your code!

### Importing code from `lab.py`

* We import our `lab.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lab import *

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import requests
import bs4
import lxml

***Note:*** While working on the lab, check the Campuswire post titled "Lab 6 Released!" for any clarifications.

## Question 1 – Practice with HTML Tags 📎

In this question, you won't write any Python code! Instead, you'll create a very basic `.html` file, named `lab06_1.html`, that satisfies the following conditions:

- It must have `<title>` and `<head>` tags.
- It must also have `<body>` tags. Within the `<body>` tags, it must have:
    - At least two headers.
    * At least three images.
        - At least one image must be a local file.
        - At least one image must be linked to online source.
        - At least one image has to have default text when it cannot be displayed.
    * At least three references (hyperlinks) to different web pages.
    * At least one table with two rows and two columns.
    

Make sure to save your file as `lab06_1.html`, and save it in the same directory as `lab.py`. **When submitting this homework to Gradescope, make sure to also upload `lab06_1.html` along with the local image that you embedded in your site.** You can upload multiple files to Gradescope at a time.
   

***Notes:*** 
- You can write and view basic HTML with a Jupyter Notebook, using either a Markdown cell or by using the `IPython.display.HTML` function (which takes in a string of HTML and renders it).
- If you write your HTML code within a Jupyter Notebook, you should later copy your code into a text editor and save it with the `.html` extension. You could also write your HTML in a text editor directly.
- Be sure to open your final `.html` file in a browser and make sure it looks correct on its own.

In [ ]:
# Don't delete this cell!
question1()

In [ ]:
grader.check("q1")

## Question 2 – Scraping an Online Bookstore 📚

Browse through the following fake online bookstore: http://books.toscrape.com/. This website is meant for toying with scraping.

Your job is to scrape the website, collecting data on all books that have:
- **_at least_ a four-star rating**, and
- **a price _strictly_ less than £50**, and 
- **belong to specific categories** (more details below). 

You will extract the information into a DataFrame that looks like the one below.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>UPC</th>
      <th>Product Type</th>
      <th>Price (excl. tax)</th>
      <th>Price (incl. tax)</th>
      <th>Tax</th>
      <th>Availability</th>
      <th>Number of reviews</th>
      <th>Category</th>
      <th>Rating</th>
      <th>Description</th>
      <th>Title</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>e10e1e165dc8be4a</td>
      <td>Books</td>
      <td>Â£22.60</td>
      <td>Â£22.60</td>
      <td>Â£0.00</td>
      <td>In stock (19 available)</td>
      <td>0</td>
      <td>Default</td>
      <td>Four</td>
      <td>For readers of Laura Hillenbrand's Seabiscuit...</td>
      <td>The Boys in the Boat: Nine Americans...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>c2e46a2ee3b4a322</td>
      <td>Books</td>
      <td>Â£25.27</td>
      <td>Â£25.27</td>
      <td>Â£0.00</td>
      <td>In stock (19 available)</td>
      <td>0</td>
      <td>Romance</td>
      <td>Five</td>
      <td>A Michelin two-star chef at twenty-eight, Violette...</td>
      <td>Chase Me (Paris Nights #2)</td>
    </tr>
    <tr>
      <th>2</th>
      <td>00bfed9e18bb36f3</td>
      <td>Books</td>
      <td>Â£34.53</td>
      <td>Â£34.53</td>
      <td>Â£0.00</td>
      <td>In stock (19 available)</td>
      <td>0</td>
      <td>Romance</td>
      <td>Five</td>
      <td>No matter how busy he keeps himself...</td>
      <td>Black Dust</td>
    </tr>
  </tbody>
</table>

To do so, implement the following functions.

#### `extract_book_links`

Create a function `extract_book_links` that takes in the content of a page that contains book listings as a **string of HTML**, and returns a **list** of URLs of book-specific pages for all books with **_at least_ a four-star rating and a price _strictly_ less than £50**. 

<br>

#### `get_product_info`

Create a function `get_product_info` that takes in the content of a book-specific page as a **string of HTML**, and a list `categories` of book categories. If the input book is in the list of `categories`, `get_product_info` should return a dictionary corresponding to a row in the DataFrame in the image above (where the keys are the column names and the values are the row values). If the input book is not in the list of `categories`, return `None`.

<br>

#### `scrape_books`

Finally, put everything together. Create a function `scrape_books` that takes in an integer `k` and a list `categories` of book categories. `scrape_books` should use `requests` to scrape the first `k` pages of the bookstore and return a DataFrame of only the books that have 
- **_at least_ a four-star rating**, and
- **a price _strictly_ less than £50**, and
- **a category that is in the list `categories`**.

<br>

***Notes:***
- The first page of the bookstore is at http://books.toscrape.com. Subsequent pages can be found by clicking the "Next" button at the bottom of the page.
- When instantiating `bs4.BeautifulSoup` objects, use the optional argument `features='lxml'` to suppress any warnings.
- Only `scrape_books` needs to make a `GET` request.
- `scrape_books` should run in under 180 seconds on the entire bookstore (`k = 50`).
- Don't worry about typecasting, i.e. it's fine if `'Number of reviews'` is not stored as type `int`. Also, don't worry if you run into encoding errors in your price columns (as the example DataFrame at the top of this cell contains).

In [ ]:
# don't delete this cell, but do run it -- it is needed for the autograder tests

# doctest for extract_book_links 
extract_book_links_fp = os.path.join('data', 'products.html')
extract_book_out = extract_book_links(
    open(extract_book_links_fp, encoding='utf-8').read()
)
extract_book_url = 'scarlet-the-lunar-chronicles-2_218/index.html'

# doc tests for get product info
get_product_info_fp = os.path.join('data', 'Frankenstein.html')
get_product_info_out = get_product_info(
    open(get_product_info_fp, encoding='utf-8').read(), ['Default']
)

# doc test for scrape books 
scrape_books_out = scrape_books(1, ['Mystery'])

In [ ]:
grader.check("q2")

## Question 3 – API Requests 🤑

You trade stocks as a hobby. As an avid `pandas` coder, you decide to calculate statistics of your favorite stocks by pulling data from a public API. The API we will work with can be found at https://financialmodelingprep.com/developer/docs/#Stock-Historical-Price. Specifically, we will use the "**Historical Daily Prices with change and volume interval**" endpoint (search for it at the linked page).

Some relevant definitions:
- Ticker: A short code that refers to a stock. For example, Apple's ticker is AAPL and Ford's ticker is F. 
- Open: The price of a stock at the beginning of a trading day.
- Close: The price of a stock at the end of a trading day.
- Volume: The total number of shares traded in a day.
- Percent change: The difference in price with respect to the original price, as a percentage.

To make requests to the aforementioned API, you will need an API key. In order to get one, you will need to make an account at the website. Once you've signed up, you can use the API key that comes with the free plan. It has a limit of 250 requests per day, which should be more than enough. You will have to encode your API key in the URL that you make requests to; see a complete example of such a request at the right side of the [documentation](https://site.financialmodelingprep.com/developer/docs#Stock-Historical-Price).

Implement the following two functions.

#### `stock_history`

Create a function `stock_history` which takes in a string `ticker` and two integers, `year` and `month`, and returns a DataFrame containing the price history for that stock in that month. Keep all of the attributes that are returned by the API.

***Notes:***
- Read the API documentation if you get stuck!
- [`pd.date_range`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html) takes in two dates and returns a sequence of all dates between the two dates, excluding the right endpoint. How might this be helpful?
- The [`requests.get`](https://docs.python-requests.org/en/master/user/quickstart/) function returns a Response object, not the data itself. Use the `json` method on the Response object to extract the relevant JSON, as we did in Lecture 15 (you don't need to `import json` to do this).
- You can instantiate a DataFrame using a sequence of dictionaries.

<br>

#### `stock_stats`

Create a function `stock_stats` that takes in a DataFrame outputted by `stock_history` and returns a **tuple** of two numbers:
1. The percent change of the stock throughout the month as a **percentage**.
2. An estimate of the total transaction volume **in billion of dollars** for that month.

Both values in the tuple should be **strings** that contain numbers rounded to two decimal places. Add a plus or minus sign in front of the percent change, and make sure that the total transaction volume string ends in a `'B'`.

**To compute the percent change**, use the opening price on the first day of the month as the starting price and the closing price on the last day of the month as the ending price.

**To compute the total transaction volume**, assume that on any given day, the average price of a share is the midpoint of the high and low price for that day.

$$ \text{Estimated Total Transaction Volume (in dollars)} = \text{Volume (number of shares traded)} \times \text{Average Price} $$

For example, suppose there are only three days in March- March 1st, March 2nd, and March 3rd.

If BYND (Beyond Meat) opens at \\$4 on March 1st and closes at \\$5 on March 3rd, its percent change for the month of March is $$\frac{\$5-\$4}{\$4} = +25.00\%$$

Suppose the high and low prices and volumes of BYND on each day are given below.
- March 1st: high \\$5, low \\$3, volume 500 million (0.5 billion)
- March 2nd: high \\$5.5, low \\$2.5, volume 1 billion
- March 3rd: high \\$5.25, low \\$4, volume 500 million (0.5 billion)

Then, the estimated total transaction volume is
$$\frac{\$5 + \$3}{2} \cdot 0.5 B + \frac{\$5.5 + \$2.5}{2} \cdot 1 B + \frac{\$5.25 + \$4}{2} \cdot 0.5 B = 8.3125B$$

In [ ]:
# don't delete this cell, but do run it -- it is needed for the autograder tests

# doctest for stock_history
history = stock_history('BYND', 2019, 6)

# doctest for stock_stats
stats = stock_stats(history)

In [ ]:
grader.check("q3")

## Question 4 – Comment Threads 🧵

You regularly browse [Hacker News](https://news.ycombinator.com/) to keep up with the latest news in tech. One link to a Hacker News article is https://news.ycombinator.com/item?id=18344932. Note that this article has 18 comments and has a `storyid` of 18344932.

The problem now is that you don't have internet access on your phone during your morning commute to work, so you want to save the interesting stories' comment threads beforehand in a CSV. You find their [API documentation](https://github.com/HackerNews/API) and decide to get to work.

Write a function `get_comments` that takes in a `storyid` and returns a DataFrame of all the comments below the news story. You can ignore "dead" comments (you will know them when you see them). **Make sure the order of the comments in your DataFrame is from top to bottom just as you see on the website**. 

The DataFrame that `get_comments` returns should have 5 columns:
1. `'id'`: The unique ID of the comment.
2. `'by'`: The author of the comment.
3. `'text'`: The actual comment.
4. `'parent'`: The unique ID of the comment this comment is replying to.
5. `'time'`: When the comment was created (in `pd.Timestamp` format).

Some guidance:
1. The URL to make requests to is `'https://hacker-news.firebaseio.com/v0/item/{}.json'`, however the `{}` should be replaced with the ID of the article or page you are trying to access. 
2. Again, do not `import json` – instead, use the `json` method on the Response object you get back.
3. Use depth-first-search when traversing the comments tree. You will have to do this manually, since you cannot use Beautiful Soup (which is only for HTML documents, not JSON objects). [This article](https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks) will be helpful.
4. Make sure the length of your returned DataFrame is the same as value for the `'descendants'` key in the response JSON (both of which correspond to the number of comments for the story).
5. You are allowed to use loops in this function. You may also want to create at least one helper function.

In [ ]:
# don't delete this cell, but do run it -- it is needed for the autograder tests
comments_doctest = get_comments(18344932)

In [ ]:
grader.check("q4")

## Congratulations! You're done! 🏁

Submit the following three files to Gradescope:
- `lab.py`
- `lab06_1.html`
- The local image you embedded in `lab06_1.html`

Before submitting, you should ensure that all of your work is in the `lab.py` file. You can do this by running the doctests below, which will verify that your work passes the public tests **and** that your work is in the `lab.py` file. Run the cell below; you should see no output.

In [ ]:
!python -m doctest lab.py

In addition, `grader.check_all()` will verify that your work passes the public tests.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()